In [1]:
import numpy as np 
import pandas as pd 

In [2]:
train_ds = pd.read_csv('train.csv')
test_ds = pd.read_csv('test.csv')

In [3]:
train_ds.rename(columns={'Province/State':'ProvinceState', 'Country/Region':'CountryRegion'}, inplace=True)
test_ds.rename(columns={'Province/State':'ProvinceState','Country/Region':'CountryRegion'}, inplace=True)
train_ds.head()

,Id,ProvinceState,CountryRegion,Lat,Long,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,33.0,65.0,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0


In [4]:
cols_with_missing = [col for col in train_ds.columns if train_ds[col].isnull().any()]
countries_with_PSinfo = train_ds[train_ds['ProvinceState'].isna()==False]['CountryRegion'].unique()

In [5]:
from datetime import date
d0 = [int(ele) for ele in min(train_ds.Date).split("-")]
d0 = date(d0[0], d0[1], d0[2])
for i in range(0, len(train_ds.Date)):
    train_ds.Date[i] = (date(int(train_ds.Date[i].split("-")[0]), int(train_ds.Date[i].split("-")[1]), int(train_ds.Date[i].split("-")[2])) - d0).days

C:\Users\behlul\AppData\Local\Temp\ipykernel_27716\644597780.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ds.Date[i] = (date(int(train_ds.Date[i].split("-")[0]), int(train_ds.Date[i].split("-")[1]), int(train_ds.Date[i].split("-")[2])) - d0).days


In [6]:
test_ds_cp = test_ds.copy()
for i in range(0, len(test_ds.Date)):
    test_ds_cp.Date[i] = (date(int(test_ds.Date[i].split("-")[0]), int(test_ds.Date[i].split("-")[1]), int(test_ds.Date[i].split("-")[2])) - d0).days

C:\Users\behlul\AppData\Local\Temp\ipykernel_27716\1187104029.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_ds_cp.Date[i] = (date(int(test_ds.Date[i].split("-")[0]), int(test_ds.Date[i].split("-")[1]), int(test_ds.Date[i].split("-")[2])) - d0).days


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
models_poly_cc = {}
models_lin_cc = {}
models_poly_f = {}
models_lin_f = {}
for country in train_ds.CountryRegion.unique():
    if country in countries_with_PSinfo:
        for country_with_PSinfo in countries_with_PSinfo:
            PSs = train_ds[train_ds['CountryRegion'] == country]['ProvinceState'].unique()
            for PS in PSs:
                PS_ds_cc = train_ds.loc[(train_ds.CountryRegion == country) & (train_ds.ProvinceState == PS)]
                PS_ds_f = train_ds.loc[(train_ds.CountryRegion == country) & (train_ds.ProvinceState == PS)]
                PS_X_cc = PS_ds_cc.iloc[:, 5:6].values
                PS_X_f = PS_ds_f.iloc[:, 5:6].values
                PS_y_cc = PS_ds_cc.iloc[:, 6].values 
                PS_y_f = PS_ds_f.iloc[:, 7].values
                poly_reg = PolynomialFeatures(degree = 7)
                X_cc_poly = poly_reg.fit_transform(PS_X_cc)
                poly_reg.fit(X_cc_poly, PS_y_cc)
                X_f_poly = poly_reg.fit_transform(PS_X_f)
                poly_reg_f = PolynomialFeatures(degree = 7)
                poly_reg_f.fit(X_f_poly, PS_y_f)
                linear_reg = LinearRegression()
                linear_reg.fit(X_cc_poly, PS_y_cc.reshape(-1,1))
                linear_reg_f = LinearRegression()
                linear_reg_f.fit(X_f_poly, PS_y_f.reshape(-1,1))
                models_poly_cc[PS] = poly_reg
                models_lin_cc[PS] = linear_reg
                models_poly_f[PS] =  poly_reg_f
                models_lin_f[PS] = linear_reg_f
    else:
        country_ds_cc = train_ds.loc[(train_ds.CountryRegion == country)]
        country_ds_f = train_ds.loc[(train_ds.CountryRegion == country)]
        country_X_cc = country_ds_cc.iloc[:, 5:6].values
        country_X_f = country_ds_f.iloc[:, 5:6].values
        country_y_cc = country_ds_cc.iloc[:, 6].values 
        country_y_f = country_ds_f.iloc[:, 7].values
        poly_reg = PolynomialFeatures(degree = 7)
        X_cc_poly = poly_reg.fit_transform(country_X_cc)
        poly_reg.fit(X_cc_poly, country_y_cc)
        X_f_poly = poly_reg.fit_transform(country_X_f)
        poly_reg_f = PolynomialFeatures(degree = 7)
        poly_reg_f.fit(X_f_poly, country_y_f)
        linear_reg = LinearRegression()
        linear_reg.fit(X_cc_poly, country_y_cc.reshape(-1,1))
        linear_reg_f = LinearRegression()
        linear_reg_f.fit(X_f_poly, country_y_f.reshape(-1,1))
        models_poly_cc[country] = poly_reg
        models_lin_cc[country] = linear_reg
        models_poly_f[country] =  poly_reg_f
        models_lin_f[country] = linear_reg_f

In [8]:
test_ds_cp['ProvinceCountry'] = np.where(test_ds_cp.ProvinceState.isnull(), test_ds_cp.CountryRegion, test_ds_cp.ProvinceState)
test_ds_cp
test_preds_cc =  []
test_preds_f = []

for index, row in test_ds_cp.iterrows():
    poly_predict_cc = models_poly_cc[row['ProvinceCountry']].fit_transform([[row['Date']]])
    predict_cc = models_lin_cc[row['ProvinceCountry']].predict(poly_predict_cc)
    poly_predict_f = models_poly_f[row['ProvinceCountry']].fit_transform([[row['Date']]])
    predict_f = models_lin_f[row['ProvinceCountry']].predict(poly_predict_f)
    test_preds_cc.append(abs(predict_cc[0,0]))
    test_preds_f.append(abs(predict_f[0,0]))

In [9]:
test_ds_out = pd.DataFrame({'ForecastId' : test_ds['ForecastId'], 'ConfirmedCases' : test_preds_cc, 'Fatalities' : test_preds_f})
test_ds_out.to_csv('submission.csv', index = False)